In [6]:
import numpy as np

from Bio.SeqUtils.IsoelectricPoint import IsoelectricPoint as IP
from Bio.PDB.Polypeptide import PPBuilder
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import freesasa
import Bio.PDB as PDB 
import Bio.PDB.DSSP as DSSP 

# atomium is a molecular modeller and file parser, capable of reading from and writing to .pdb, .cif and .mmtf files
import atomium

In [5]:
def calc_length(filenames):

    prot_lengths = []

    for filename in filenames:
        # parse the pdb file
        p = PDB.PDBParser(QUIET=True)
        s = p.get_structure("", filename)

        # get the sequence length
        seq = 0
        for chain in s.get_chains():
            seq += len([_ for _ in chain.get_residues() if PDB.is_aa(_)])

        # save into numpy sheet rsa
        prot_lengths.append(seq)

    return prot_lengths

In [ ]:
def calc_surface(filenames):

    surfaces = []
    for filename in filenames:

        # get the surface area
        structure = freesasa.Structure(filename)
        result = freesasa.calc(structure)
        area_classes = freesasa.classifyResults(result, structure)
        
        # save this into numpy sheet result.totalArea() 
        surface = result.totalArea()
        surfaces.append(surface)

    return surfaces

In [ ]:
def ss_depth(filenames):

    frac_mod_beta_list = []
    frac_mod_alfa_list = []
    frac_exp_alfa_list = []

    for filename in filenames:
        p = PDB.PDBParser()
        structure = p.get_structure("", filename)
            
        model = structure[0]
        dssp = DSSP(model, filename)

        # DSSP data is accessed by a tuple (chain_id, res_id)
        all_residues = list(dssp.keys())

        # get a list with ASA values of all residues
        asa = [dssp[i][3] for i in all_residues]

        # 3 categories, like in paper
        burried = [0 if i <= 0.2 else 2 if i >= 0.5 else 1 for i in asa]

        # get a list with secondary structure of all residues (Q8)
        secondary_q8 = [dssp[i][2] for i in all_residues]

        # map the Q8 to Q3
        # helix = H, G, I
        # beta = B, E 
        # loop = rest 
        # 0 is alpha, 1 is beta, 2 is coil
        secondary_q3 = [0 if i in ['H', 'G', 'I'] else 1 if i in ['B', 'E'] else 2 for i in secondary_q8]

        # get the total number of sheets and helices
        count_helices = secondary_q3.count(0)
        count_sheets = secondary_q3.count(1)

        if count_sheets == 0:
            frac_mod_beta_list.append(0)
        else:
            # calculate fraction of moderately buried beta residues
            mod_beta = 0

            for i in range(len(burried)): 
                if burried[i] == 1 and secondary_q3[i] == 1:
                    mod_beta += 1
                
            frac_mod_beta = mod_beta / count_sheets
            frac_mod_beta_list.append(frac_mod_beta)

        if count_helices == 0:
            frac_mod_alfa_list.append(0)
            frac_exp_alfa_list.append(0)
        else:
            # calc fraction of moderately buried alfa residues
            mod_alfa = 0

            for i in range(len(burried)): 
                if burried[i] == 1 and secondary_q3[i] == 0:
                    mod_alfa += 1
                
            frac_mod_alfa = mod_alfa / count_helices
            frac_mod_alfa_list.append(frac_mod_alfa)

            # calc fraction of exposed a residues
            exp_alfa = 0

            for i in range(len(burried)): 
                if burried[i] == 2 and secondary_q3[i] == 0:
                    exp_alfa += 1
                
            frac_exp_alfa = exp_alfa / count_helices
            frac_exp_alfa_list.append(frac_exp_alfa)

    return frac_mod_beta_list, frac_mod_alfa_list, frac_exp_alfa_list

In [ ]:
def feature_calc(filenames):
    """Funciton that calculats the , charge at pH 7, isoelectric point, aromaticity, count of aromaic acids, 
    flexibility and instability for a list of proteins"""
    prot_charge = []
    pi = []
    aromatic_count = []
    aromaticity = []
    flex = []
    weight = []
    inst = []

    for file in filenames:

        # parse the pdb file
        p = PDB.PDBParser(QUIET=True)
        s = p.get_structure(file, file)

        PolypeptideBuilder = PPBuilder()

        for pp in PolypeptideBuilder.build_peptides(s):
            query_chain = pp.get_sequence()

        # calculate iso-electric point etc.
        protein = IP(query_chain)
        prot_charge.append(protein.charge_at_pH(7.0))
        pi.append(protein.pi())

        # calculate molecular weight
        X = ProteinAnalysis(str(query_chain))
        # print(str(query_chain) + "\n")
        aromatic_count.append(X.count_amino_acids()['F'] + X.count_amino_acids()['W'] + X.count_amino_acids()['Y'])

        # Calculate aromaticity
        aromaticity.append(X.aromaticity())

        # molecular weight
        weight.append(X.molecular_weight())

        # flexibility calculated from aa. sequence
        flex.append(X.flexibility())

        # stability from aa. sequence
        inst.append(X.instability_index())

    return prot_charge, pi, aromatic_count, aromaticity, weight, inst, flex

In [ ]:
def radius_of_gyration(filenames):
    """Calculates the radius of gyration of each pdb structure and returns a list with the result."""
    
    rg_list = []
    for filename in filenames:
        pdb = atomium.open(filename)
        rg = pdb.model.radius_of_gyration
        rg_list.append(rg)
    
    return rg_list

In [ ]:
def get_average_bfactor(files):
    """Function that calculates the average b factor for a list of proteins"""
    av_b_factors = []

    for file in files:
        # parse the pdb file
        p = PDB.PDBParser(QUIET=True)
        s = p.get_structure(file, file)

        nratoms, sumbfactor = 0, 0.0

        for model in s:
            for chain in model:
                for residue in chain:
                    if PDB.is_aa(residue.get_resname()):
                        for atom in residue:
                            sumbfactor += atom.get_bfactor()
                            nratoms += 1

        avgbfactor = sumbfactor / nratoms
        av_b_factors.append(avgbfactor)

    return av_b_factors